In [24]:
import shapefile
DIR = "/Users/damoncrockett/Desktop/Twitter_Global_Viz/"
FILE = "TM_WORLD_BORDERS-0.3/TM_WORLD_BORDERS-0.3.shp"
polygons_sf = shapefile.Reader(DIR+FILE)
records = polygons_sf.records()
polygon_shapes = polygons_sf.shapes()
polygon_points = [q.points for q in polygon_shapes]

In [25]:
# polygon from http://thematicmapping.org/downloads/world_borders.php

In [26]:
china_points = polygon_points[29]

In [27]:
lons = [item[0] for item in china_points] 
lats = [item[1] for item in china_points]

In [28]:
import pandas as pd

In [29]:
df = pd.DataFrame(lons,columns=['lon'])

In [30]:
df['lat'] = lats

In [32]:
df.to_csv("./china_points.csv",index=False)

In [34]:
# send to R, check on a map...looks good! Over 9,000 points!

In [39]:
tweets = pd.read_csv("/Users/damoncrockett/Desktop/Twitter_Global_Viz/twitter_metadata_2014_lean.csv")

In [36]:
from shapely.geometry import Point, Polygon

In [40]:
polygon_list = [Polygon(china_points)]

In [41]:
from rtree import index
idx = index.Index()
count = -1
for item in polygon_list:
    count +=1
    idx.insert(count, item.bounds)

In [43]:
print_points = [100000,200000,400000,800000,1000000,10000000,50000000,100000000]

In [44]:
m = len(tweets)
countries = []
for i in range(m):
    if i in print_points:
        print i
    tmp = 'outside'
    pt = (tweets.lon[i],tweets.lat[i])
    for j in idx.intersection(pt):
        if Point(pt).within(polygon_list[j]):
            tmp = "china"
            break
    countries.append(tmp)

100000
200000
400000
800000
1000000
10000000
50000000
100000000


In [45]:
tweets['country'] = countries

In [46]:
tweets.country.value_counts()

outside    133775818
china          59151
dtype: int64

In [47]:
tmp = tweets[tweets.country=='china']

In [48]:
tmp.to_csv("./china_2014.csv",index=False)